In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import resgen as rg
import os

rgc = rg.connect(os.getenv("RESGEN_USER"), os.getenv("RESGEN_PASSWORD"))

In [6]:
project = rgc.find_or_create_project('Krietenstein et al. 2019', group='paper-data')

In [18]:
project.sync_dataset(
    'http://proteome.mit.edu/~nezar/data/krietenstein2019/U54-H1ESC4DN-FA-DSG-MNase__hg38.mapq30.shifted.100.mcool',
    name='H1ESC',
    datatype='matrix', filetype='cooler'
)

E8aSzhwA: H1ESC

In [19]:
project.sync_dataset(
    'http://proteome.mit.edu/~nezar/data/krietenstein2019/U54-HFFc64DN-FA-DSG-MNase__hg38.mapq30.shifted.100.mcool',
    name='HFFc6',
    datatype='matrix',
    filetype='cooler'
)

E3W9nJOh: HFFc6

In [20]:
ds_dict = dict([(d.name,d) for d in project.list_datasets()])
ds_dict

{'H1ESC': E8aSzhwA: H1ESC, 'HFFc6': E3W9nJOh: HFFc6}

In [25]:
chromsizes = rgc.find_datasets(
  datatype='chromsizes', assembly='hg38'
)[0]
chrominfo = rgc.get_chrominfo(chromsizes)

gene_annotations = rgc.find_datasets(
    datatype='gene-annotations', assembly='hg38'
)[0]

## Figure 1

In [65]:
from higlass.client import Track, View, ViewportProjection
import higlass

def create_figure1_views(
    heatmap_ds,
    locations
):
    views = []
    proj_v = None
    view_width = 12 // len(locations)
    
    for i,location in enumerate(locations):
        v = View([
                heatmap_ds.hg_track(height=100),
                chromsizes.hg_track(track_type='vertical-chromosome-labels',
                                    position='left'),
            ], width=view_width, x=i*view_width,
            initialXDomain=chrominfo.to_abs_range(*location),
        )
        views += [v]
    
    return views

views = create_figure1_views(
    ds_dict['HFFc6'],
    [
        ('chr3', 0, 55e6),
        ('chr5', 0, 22e6),
        ('chr5', 1, 6e6),
        ('chr5', 3, 5e6),
    ]
)

(d,s,v) = higlass.display(views)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'bFG75xPJSvufejg_d4qGAA', 'tracks': {'top': [], '…

## Figure 3

In [55]:
from higlass.client import Track, View, ViewportProjection
import higlass

v3 = View([
        ds_dict['H1ESC'].hg_track(height=150),
        chromsizes.hg_track(track_type='vertical-chromosome-labels',
                            position='left'),
    ], width=4, x=8,
    initialXDomain=chrominfo.to_abs_range('chr5', 62.38e6, 62.44e6),
)
proj_v3 = ViewportProjection(v3)

v2 = View([
        ds_dict['H1ESC'].hg_track(height=150) + proj_v3,
        chromsizes.hg_track(track_type='vertical-chromosome-labels',
                            position='left')
    ], width=4, x=4,
    initialXDomain=chrominfo.to_abs_range('chr5', 62.25e6, 62.45e6),
)
proj_v2 = ViewportProjection(v2)

v1 = View([
        ds_dict['H1ESC'].hg_track(height=150) + proj_v2,
        chromsizes.hg_track(track_type='vertical-chromosome-labels',
                            position='left')
    ], width=4, x=0,
    initialXDomain=chrominfo.to_abs_range('chr5', 62.25e6, 62.8e6),
)

(d,s,v) = higlass.display([v1, v2, v3])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'a7voIVHKTvi7S94LMHQCLQ', 'tracks': {'top': [], '…

In [45]:
project.sync_viewconf(v, "Figure 3")

In [53]:
from higlass.client import Track, View
import higlass

ds = rgc.find_datasets('ENCFF696JDC')[0]

(d,s,v) = higlass.display([View([ds.hg_track()], initialXDomain=[0,1e9])])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'ZuRtfibZQGmPcEMNtuMobA', 'tracks': {'top': [{'ty…